## Import libraries

In [20]:
from PIL import Image
import numpy as np
import random
import math

## Read Image

In [7]:
def read_pnm(file_name):
  with open(file_name, 'r') as f:
    vals = f.read().split()
  w = int(vals[1])
  h = int(vals[2])
  if vals[0] == 'P2': # Grayscale
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w))
  elif vals[0] == 'P3': # RGB
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w, 3))
  else:
    return None
  
  return pixels

In [8]:
def show_pnm(file_name):
  with open(file_name, 'r') as f:
    vals = f.read().split()
  w = int(vals[1])
  h = int(vals[2])
  if vals[0] == 'P2': # Grayscale
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w))
  elif vals[0] == 'P3': # RGB
    pixels = np.array(vals[4:], dtype=np.uint8).reshape((h, w, 3))
  else:
    return None
  
  return Image.fromarray(pixels)

In [2]:
def WritePnm(file_name, pixels, h, w , numchannels):
  f = open(file_name, "a")
  if (numchannels == 3):
    f.write('P3\n')
  else:
    f.write('P2\n')
  f.write(str(w) + ' ')
  f.write(str(h) + '\n')
  f.write('255\n')
  for i in pixels:
    for j in i:
      for k in j:
        f.write(str(k) + ' ')
    f.write('\n')

In [12]:
srcImage = read_pnm('in.pnm')
print(srcImage[0][0])

[  0  89 152]


In [32]:
#Hàm khởi tạo tâm cho từng Clusters
def initClusters(srcImage, k=2) :
    # vector clusters chứa kết quả trả về
    centers = []

    # Lấy giá trị kích thước ảnh
    shape = srcImage.shape
    rows = shape[0]
    cols = shape[1]

    # Khởi tạo theo số cụm cluster đã khai báo
    for i in range(k):
        center = []
        rand_x = random.randint(0,rows)
        rand_y = random.randint(0,cols)
        center = srcImage[rand_x, rand_y] 

        # Thêm cluster thứ index vào vector
        centers.append([center,i])

    # Trả về vector cluster đã được khởi tạo
    return centers

# Hàm tính khoảng khác biệt giữa 2 pixel bằng công thức Euclid
def EuclideanDistance (pixel1, pixel2, dim):
  result = 0
  for i in range(0, dim) :
    result += pow(pixel1[i] - pixel2[i],2)
  result = round(math.sqrt(result))
  return result

#Lấy giá trị trung bình của cluster đối với ảnh có 1 kênh màu
def meanCluster(srcImage, pixels, label):
    temp = []
    # Lấy kích thước của ảnh đầu vào
    shape = srcImage.shape
    print(shape)
    rows = shape[0]
    cols = shape[1]
    dim = shape[2]
    # Duyệt qua ảnh đầu vào 
    for row in range(0,rows) :
        for col  in range(0,cols) :
            if (pixels[row][col] == label):
                temp.append(srcImage[row, col])
    
    temp_size = len(temp)
    # đối với ảnh 1 kênh màu
    if (dim == 1):       
		# Tính tổng giá trị của vector temp
      sum = 0
      for k in range(0,temp_size) :
        sum += temp[k]
        # Trả về giá trị trung bình của vector temp
      if (temp_size < 1):
        return 255
      else:
        return (sum / temp_size)
     
     # Đối với ảnh 3 kênh màu
    elif (dim ==3):
      sumB = 0
      sumG = 0
      sumR = 0
      for k in range(0,temp_size):
        sumB += temp[k][0]
        sumG += temp[k][1]
        sumR += temp[k][2]
        # Trả về giá trị trung bình của vector temp
      if (temp_size < 1):
        return 255
      else:
        return [sumB/temp_size, sumG/temp_size, sumR/temp_size] 

In [33]:
def apply(srcImage):
  dstImage = srcImage;

  clusters = initClusters(srcImage);

  shape = srcImage.shape
  rows = shape[0]
  cols = shape[1]

  pixels = [[-1]*cols for i in range(rows)]

  index = 0
  prevs = []

  while (True):
    prevs = clusters

    for row in range(rows):
      for col in range(cols):
        minDiff = 0
        minDiff = EuclideanDistance(srcImage[row][col], clusters[0])

        diff = 0
        label = 0

        for (k in range(1, len(clusters))):
          diff = EuclideanDistance(srcImage[row][col], clusters[k])
          if (diff < minDiff):
            minDiff = diff
            label = k

        pixels[row][col] = clusters[label].label

    for (k in range(0, len(clusters))):
      clusters[k].value = meanCluster(srcImage, pixels, clusters[k].label)


    cambiati = 0
    for (k in range(0, len(clusters))):
      if (prevs[k].value != clusters[k].value):
        cambiati += 1

    if (cambiati == 0):
      break

    index += 1

    for (row in range(0, shape[0])):
      for (col in range(0, shape[1])):
        dstImage[row][col] = clusters[pixels[row][col]].value

In [34]:
apply(srcImage)